In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pycaret[full] #catboost, xgboost등 사용위해선 full로 설치

     |████████████████████████████████| 301 kB 6.9 MB/s 
     |████████████████████████████████| 261 kB 58.7 MB/s 
     |████████████████████████████████| 6.8 MB 55.2 MB/s 
     |████████████████████████████████| 114 kB 51.4 MB/s 
     |████████████████████████████████| 271 kB 53.8 MB/s 
     |████████████████████████████████| 15.6 MB 42.1 MB/s 
     |████████████████████████████████| 1.3 MB 53.2 MB/s 
     |████████████████████████████████| 86 kB 7.3 MB/s 
     |████████████████████████████████| 2.0 MB 42.1 MB/s 
     |████████████████████████████████| 167 kB 48.7 MB/s 
     |████████████████████████████████| 56 kB 4.1 MB/s 
     |████████████████████████████████| 1.7 MB 46.5 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 871 kB 51.2 MB/s 
     |████████████████████████████████| 13.4 MB 28.1 MB/s 
     |████████████████

In [3]:
!pip install xgboost
!pip install catboost
!pip install lightgbm

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GridSearchCV, cross_val_score, cross_validate
from sklearn.metrics import roc_curve, auc, accuracy_score
from sklearn.ensemble import RandomForestClassifier
from pycaret.classification import *
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier

/usr/local/lib/python3.7/dist-packages/distributed/config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


In [48]:
train = pd.read_csv('/content/drive/MyDrive/YBIGTA/DA 시니어 2차 프로젝트/airline_dataset/train.csv')
train = train.drop('id', axis=1)
test = pd.read_csv('/content/drive/MyDrive/YBIGTA/DA 시니어 2차 프로젝트/airline_dataset/test.csv')
test = test.drop('id', axis=1)
train

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes,target
0,Female,disloyal Customer,22,Business travel,Eco,1599,3,0,3,3,4,3,4,4,5,4,4,4,5,4,0,0.0,0
1,Female,Loyal Customer,37,Business travel,Business,2810,2,4,4,4,1,4,3,5,5,4,2,1,5,2,18,18.0,0
2,Male,Loyal Customer,46,Business travel,Business,2622,1,1,1,1,4,5,5,4,4,4,4,5,4,3,0,0.0,1
3,Female,disloyal Customer,24,Business travel,Eco,2348,3,3,3,3,3,3,3,3,2,4,5,3,4,3,10,2.0,0
4,Female,Loyal Customer,58,Business travel,Business,105,3,3,3,3,4,4,5,4,4,4,4,4,4,5,0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,Male,Loyal Customer,30,Personal Travel,Eco,2377,1,5,1,1,1,4,4,1,3,5,3,4,2,4,211,225.0,0
2996,Female,disloyal Customer,24,Business travel,Eco,1643,2,4,3,4,5,3,5,5,2,2,4,1,3,5,20,13.0,0
2997,Female,disloyal Customer,22,Business travel,Eco,1442,2,2,2,3,4,2,4,4,3,2,3,4,3,4,64,67.0,0
2998,Female,disloyal Customer,33,Business travel,Business,2158,2,2,2,5,4,2,4,4,5,2,5,5,5,4,0,3.0,0


In [49]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 23 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Gender                             3000 non-null   object 
 1   Customer Type                      3000 non-null   object 
 2   Age                                3000 non-null   int64  
 3   Type of Travel                     3000 non-null   object 
 4   Class                              3000 non-null   object 
 5   Flight Distance                    3000 non-null   int64  
 6   Seat comfort                       3000 non-null   int64  
 7   Departure/Arrival time convenient  3000 non-null   int64  
 8   Food and drink                     3000 non-null   int64  
 9   Gate location                      3000 non-null   int64  
 10  Inflight wifi service              3000 non-null   int64  
 11  Inflight entertainment             3000 non-null   int64

In [50]:
continuous_features = ['Age', 'Flight Distance', 'Departure Delay in Minutes', 'Arrival Delay in Minutes']
ordinal_features = ['Seat comfort', 'Departure/Arrival time convenient', 'Food and drink', 'Gate location', 
           'Inflight wifi service', 'Inflight entertainment', 'Online support', 'Ease of Online booking', 
           'On-board service', 'Leg room service', 'Baggage handling', 'Checkin service', 'Cleanliness', 'Online boarding']
categorical_features = ['Gender', 'Customer Type', 'Type of Travel', 'Class']

In [51]:
for f in categorical_features:
    print(train[f].value_counts())

Male      1502
Female    1498
Name: Gender, dtype: int64
Loyal Customer       2489
disloyal Customer     511
Name: Customer Type, dtype: int64
Business travel    2053
Personal Travel     947
Name: Type of Travel, dtype: int64
Business    1434
Eco         1337
Eco Plus     229
Name: Class, dtype: int64


## 범주형변수 인코딩

In [52]:
''' # 성별 원핫인코딩 - 성별도 값 맵핑하는게 더나음
def cat_1hot(data):
  data[pd.get_dummies(data['Gender']).columns] = pd.get_dummies(data['Gender']).values
  data.drop('Gender', axis=1, inplace=True)
  col1 = data.columns[-2:].to_list()
  col2 = data.columns[:-2].to_list()
  cols = col1+col2
  data = data[cols]
  return data

test = cat_1hot(test)
train = cat_1hot(train) '''

" # 성별 원핫인코딩 - 성별도 값 맵핑하는게 더나음\ndef cat_1hot(data):\n  data[pd.get_dummies(data['Gender']).columns] = pd.get_dummies(data['Gender']).values\n  data.drop('Gender', axis=1, inplace=True)\n  col1 = data.columns[-2:].to_list()\n  col2 = data.columns[:-2].to_list()\n  cols = col1+col2\n  data = data[cols]\n  return data\n\ntest = cat_1hot(test)\ntrain = cat_1hot(train) "

In [53]:
# 나머지 범주형변수들 등급 고려하여 맵핑
def cat_enc(data):
  data['Type of Travel'] = data['Type of Travel'].map({"Business travel":1, "Personal Travel":0})
  data['Class'] = data['Class'].map({"Business":2, "Eco Plus":1, "Eco":0})
  data['Customer Type'] = data['Customer Type'].map({"Loyal Customer":1, "disloyal Customer":0})
  data['Gender'] = data['Gender'].map({'Female':1, 'Male':0}) #성별 원핫,레이블인코딩 할지 결정!
  return data

test = cat_enc(test)
train = cat_enc(train)

In [54]:
train.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes,target
0,1,0,22,1,0,1599,3,0,3,3,4,3,4,4,5,4,4,4,5,4,0,0.0,0
1,1,1,37,1,2,2810,2,4,4,4,1,4,3,5,5,4,2,1,5,2,18,18.0,0
2,0,1,46,1,2,2622,1,1,1,1,4,5,5,4,4,4,4,5,4,3,0,0.0,1
3,1,0,24,1,0,2348,3,3,3,3,3,3,3,3,2,4,5,3,4,3,10,2.0,0
4,1,1,58,1,2,105,3,3,3,3,4,4,5,4,4,4,4,4,4,5,0,0.0,1


## 응답중 0값 대체, 다중공선성 제거

In [55]:
for col in ordinal_features:
  if (train[col]==0).sum() == 0:
    continue
  print(f'{col} : 0이 \'{(train[col]==0).sum()}\'개 있습니다.\n')

Seat comfort : 0이 '106'개 있습니다.

Departure/Arrival time convenient : 0이 '154'개 있습니다.

Food and drink : 0이 '129'개 있습니다.

Inflight wifi service : 0이 '2'개 있습니다.

Inflight entertainment : 0이 '73'개 있습니다.

Ease of Online booking : 0이 '1'개 있습니다.

Leg room service : 0이 '11'개 있습니다.

Online boarding : 0이 '1'개 있습니다.



In [56]:
corr = train.corr()

for col in ordinal_features:
  if (train[col]==0).sum() == 0:
    continue
  print(f'======================={col}========================')
  print(corr[col].sort_values(ascending=False)[[1,2,3]])

=======================Seat comfort========================
Food and drink                       0.726269
Departure/Arrival time convenient    0.440134
Inflight entertainment               0.425375
Name: Seat comfort, dtype: float64
=======================Departure/Arrival time convenient========================
Gate location     0.524651
Food and drink    0.517608
Seat comfort      0.440134
Name: Departure/Arrival time convenient, dtype: float64
=======================Food and drink========================
Seat comfort                         0.726269
Gate location                        0.518372
Departure/Arrival time convenient    0.517608
Name: Food and drink, dtype: float64
=======================Inflight wifi service========================
Online boarding           0.632049
Ease of Online booking    0.602040
Online support            0.563290
Name: Inflight wifi service, dtype: float64
=======================Inflight entertainment========================
target            0.5229

0값이 많은 상위 4개(출발도착편의, 식음료, 좌석, 기내오락) 위주로 고민  
나머지는 최빈값으로 대체하기 그냥.
- Seatcomfort(106) = 식음료를 없앴으므로 최빈값, 타겟과상관관계 높진않아서(0.27) 제거한 모델과 성능 비교 꼭!
- 출발도착편의(154) = 가장많은 결측치, 타겟과 상관관계(0.15)낮고 식음료와 상관계수높음(0.52) 최빈값4 대체하거나 그냥 제거하기 >>> 제거하자
- 식음료(129) = 제거, 좌석과 높은상관관계(다중공선성), 타겟과는 보다 낮은상관관계
- 기내오락(73) = 타겟과상관계수 높은편(0.52), 최빈값대체

추가적으로, Departure Delay in Minutes도 도착지연시간과 매우 높은상관관계를 가지므로 제거

In [57]:
# 최빈값대체
def null_to_mode(data, col):
  data[col] = data[col].replace([0],[data[col].mode()])
  return data

null_to_mode(train, 'Seat comfort')
null_to_mode(train, 'Inflight entertainment')
null_to_mode(test, 'Seat comfort')
null_to_mode(test, 'Inflight entertainment')

null_to_mode(train, 'Departure/Arrival time convenient')
null_to_mode(test, 'Departure/Arrival time convenient')

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Food and drink,Gate location,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Departure Delay in Minutes,Arrival Delay in Minutes
0,1,1,61,0,0,2037,1,1,1,1,3,5,5,5,5,5,5,3,5,3,51,58.0
1,1,0,27,1,2,1846,1,1,1,4,1,1,1,1,3,4,5,4,4,1,0,0.0
2,1,1,52,1,2,1622,4,4,4,4,4,3,4,5,5,5,5,4,5,3,0,0.0
3,0,1,54,1,2,3534,4,4,4,4,5,4,5,2,2,2,2,5,2,1,0,0.0
4,1,1,41,1,0,1471,4,3,3,3,4,4,4,4,3,1,2,5,4,4,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1,1,62,1,1,54,3,1,1,1,4,3,4,3,3,3,3,3,3,1,0,0.0
1996,1,1,33,1,2,1937,4,4,4,4,3,5,5,5,5,5,5,1,5,3,95,99.0
1997,0,1,29,1,0,2398,3,5,4,4,3,3,3,3,1,2,3,4,2,3,0,3.0
1998,1,1,51,0,2,406,5,2,5,4,4,3,3,2,2,5,2,2,2,2,87,83.0


In [58]:
''' # 특성의 평균값 대체 >> 최빈값이 더 나음
def null_to_mean(data, col):
  data[col] = data[col].replace([0],[data[col].mean()])
  return data

null_to_mode(train, 'Seat comfort')
null_to_mode(train, 'Inflight entertainment')
null_to_mode(test, 'Seat comfort')
null_to_mode(test, 'Inflight entertainment') '''

" # 특성의 평균값 대체 >> 최빈값이 더 나음\ndef null_to_mean(data, col):\n  data[col] = data[col].replace([0],[data[col].mean()])\n  return data\n\nnull_to_mode(train, 'Seat comfort')\nnull_to_mode(train, 'Inflight entertainment')\nnull_to_mode(test, 'Seat comfort')\nnull_to_mode(test, 'Inflight entertainment') "

In [59]:
# 열 삭제
#train.drop(['Departure Delay in Minutes','Departure/Arrival time convenient', 'Food and drink'], axis=1, inplace = True)
#test.drop(['Departure Delay in Minutes', 'Departure/Arrival time convenient', 'Food and drink'], axis=1, inplace = True)
train.drop(['Departure Delay in Minutes', 'Food and drink'], axis=1, inplace = True)
test.drop(['Departure Delay in Minutes', 'Food and drink'], axis=1, inplace = True)

In [60]:
train

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Gate location,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Arrival Delay in Minutes,target
0,1,0,22,1,0,1599,3,4,3,4,3,4,4,5,4,4,4,5,4,0.0,0
1,1,1,37,1,2,2810,2,4,4,1,4,3,5,5,4,2,1,5,2,18.0,0
2,0,1,46,1,2,2622,1,1,1,4,5,5,4,4,4,4,5,4,3,0.0,1
3,1,0,24,1,0,2348,3,3,3,3,3,3,3,2,4,5,3,4,3,2.0,0
4,1,1,58,1,2,105,3,3,3,4,4,5,4,4,4,4,4,4,5,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0,1,30,0,0,2377,1,5,1,1,4,4,1,3,5,3,4,2,4,225.0,0
2996,1,0,24,1,0,1643,2,4,4,5,3,5,5,2,2,4,1,3,5,13.0,0
2997,1,0,22,1,0,1442,2,2,3,4,2,4,4,3,2,3,4,3,4,67.0,0
2998,1,0,33,1,2,2158,2,2,5,4,2,4,4,5,2,5,5,5,4,3.0,0


## 특성 스케일링


In [61]:
continuous_features = ['Age', 'Flight Distance', 'Arrival Delay in Minutes']

scaler = StandardScaler()
scaled_train = scaler.fit_transform(train[continuous_features])
scaled_test = scaler.transform(test[continuous_features]) #주의! 테스트데이터분포 학습하면 안됨! 훈련세트로 fit한 객체로 test셋 transform.
train[continuous_features] = pd.DataFrame(scaled_train)
test[continuous_features] = pd.DataFrame(scaled_test)
display(train.head())
display(test.head())

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Gate location,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Arrival Delay in Minutes,target
0,1,0,-1.138798,1,0,-0.373641,3,4,3,4,3,4,4,5,4,4,4,5,4,-0.352289,0
1,1,1,-0.145833,1,2,0.804446,2,4,4,1,4,3,5,5,4,2,1,5,2,0.045978,0
2,0,1,0.449945,1,2,0.621556,1,1,1,4,5,5,4,4,4,4,5,4,3,-0.352289,1
3,1,0,-1.006402,1,0,0.355003,3,3,3,3,3,3,3,2,4,5,3,4,3,-0.308037,0
4,1,1,1.244317,1,2,-1.827036,3,3,3,4,4,5,4,4,4,4,4,4,5,-0.352289,1


,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Gate location,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Arrival Delay in Minutes
0,1,1,1.442910,0,0,0.052455,1,1,1,3,5,5,5,5,5,5,3,5,3,0.931014
1,1,0,-0.807810,1,2,-0.133354,1,1,4,1,1,1,1,3,4,5,4,4,1,-0.352289
2,1,1,0.847131,1,2,-0.351266,4,4,4,4,3,4,5,5,5,5,4,5,3,-0.352289
3,0,1,0.979526,1,2,1.508769,4,4,4,5,4,5,2,2,2,2,5,2,1,-0.352289
4,1,1,0.118957,1,0,-0.498162,4,3,3,4,4,4,4,3,1,2,5,4,4,-0.352289


In [39]:
train.plot(kind='box', subplots=True, layout=(5, 5), figsize=(15, 21))
plt.show()       

ImportError: ignored

<Figure size 1080x1512 with 25 Axes>

## Log-Transformation

In [62]:
train['Flight Distance'] = np.log1p(train['Flight Distance'])
train['Arrival Delay in Minutes'] = np.log1p(train['Arrival Delay in Minutes'])
test['Flight Distance'] = np.log1p(test['Flight Distance'])
test['Arrival Delay in Minutes'] = np.log1p(test['Arrival Delay in Minutes'])

## 이상치 제거, 대체

In [63]:
def outliers_iqr(data):
    q1, q3 = np.percentile(data, [25, 75])
    # 넘파이의 값을 퍼센트로 표시해주는 함수

    iqr = q3 - q1
    lower_bound = q1 - (iqr * 1.5)
    upper_bound = q3 + (iqr * 1.5)
    
    return np.where((data > upper_bound) | (data < lower_bound))

In [64]:
def outliers_mean(data, col):
  data_index = outliers_iqr(data[col])[0]
  data.loc[data_index, col] = data[col].mean()
  return data

In [65]:
outliers_mean(test, 'Flight Distance')
outliers_mean(test, 'Arrival Delay in Minutes')
outliers_mean(train, 'Flight Distance')
outliers_mean(train, 'Arrival Delay in Minutes')

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Gate location,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Arrival Delay in Minutes,target
0,1,0,-1.138798,1,0,-0.467831,3,4,3,4,3,4,4,5,4,4,4,5,4,-0.434310,0
1,1,1,-0.145833,1,2,0.590254,2,4,4,1,4,3,5,5,4,2,1,5,2,0.044952,0
2,0,1,0.449945,1,2,0.483386,1,1,1,4,5,5,4,4,4,4,5,4,3,-0.434310,1
3,1,0,-1.006402,1,0,0.303803,3,3,3,3,3,3,3,2,4,5,3,4,3,-0.368223,0
4,1,1,1.244317,1,2,NaN,3,3,3,4,4,5,4,4,4,4,4,4,5,-0.434310,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,0,1,-0.609217,0,0,0.324410,1,5,1,1,4,4,1,3,5,3,4,2,4,-0.166393,0
2996,1,0,-1.006402,1,0,-0.401727,2,4,4,5,3,5,5,2,2,4,1,3,5,-0.066837,0
2997,1,0,-1.138798,1,0,-0.747337,2,2,3,4,2,4,4,3,2,3,4,3,4,-0.166393,0
2998,1,0,-0.410624,1,2,0.157146,2,2,5,4,2,4,4,5,2,5,5,5,4,-0.336748,0


In [66]:
train.head()

,Gender,Customer Type,Age,Type of Travel,Class,Flight Distance,Seat comfort,Departure/Arrival time convenient,Gate location,Inflight wifi service,Inflight entertainment,Online support,Ease of Online booking,On-board service,Leg room service,Baggage handling,Checkin service,Cleanliness,Online boarding,Arrival Delay in Minutes,target
0,1,0,-1.138798,1,0,-0.467831,3,4,3,4,3,4,4,5,4,4,4,5,4,-0.434310,0
1,1,1,-0.145833,1,2,0.590254,2,4,4,1,4,3,5,5,4,2,1,5,2,0.044952,0
2,0,1,0.449945,1,2,0.483386,1,1,1,4,5,5,4,4,4,4,5,4,3,-0.434310,1
3,1,0,-1.006402,1,0,0.303803,3,3,3,3,3,3,3,2,4,5,3,4,3,-0.368223,0
4,1,1,1.244317,1,2,NaN,3,3,3,4,4,5,4,4,4,4,4,4,5,-0.434310,1


## pycaret으로 간단히 확인하기

In [67]:
continuous_features = ['Age', 'Flight Distance', 'Arrival Delay in Minutes']
ordinal_features = ['Seat comfort', 'Gate location', 
           'Inflight wifi service', 'Inflight entertainment', 'Online support', 'Ease of Online booking', 
           'On-board service', 'Leg room service', 'Baggage handling', 'Checkin service', 'Cleanliness', 'Online boarding'] #출발도착편의, 식음료삭제
categorical_features = ['Gender', 'Customer Type', 'Type of Travel', 'Class'] #원핫인코딩시 젠더를 메일,피메일로 변경!

In [68]:
clf = setup(data=train, 
            target='target',
            silent = True,
            #categorical_features = categorical_features,
            #numeric_features = continuous_features,
            normalize = True,
            normalize_method = 'zscore',  #minmax, robust등 가능
            #remove_outliers = True,
            #outliers_threshold = 0.05,
            #remove_multicollinearity = True, #상관계수0.9(default)이상 피쳐제거
            #train_size = 0.8,
            n_jobs = -1)

,Description,Value
0,session_id,3662
1,Target,target
2,Target Type,Binary
3,Label Encoded,None
4,Original Data,"(3000, 21)"
5,Missing Values,True
6,Numeric Features,3
7,Categorical Features,17
8,Ordinal Features,False
9,High Cardinality Features,False


In [ ]:
best = compare_models(n_select=3, sort='Accuracy', fold=5)

IntProgress(value=0, description='Processing: ', max=84)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,10:25:33
Status,. . . . . . . . . . . . . . . . . .,Fitting 5 Folds
Estimator,. . . . . . . . . . . . . . . . . .,CatBoost Classifier


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
xgboost,Extreme Gradient Boosting,0.9190,0.9763,0.9243,0.9311,0.9275,0.8358,0.8363,10.612
lightgbm,Light Gradient Boosting Machine,0.9142,0.9777,0.9218,0.9254,0.9233,0.8260,0.8267,0.106
gbc,Gradient Boosting Classifier,0.9119,0.9726,0.9260,0.9177,0.9216,0.8209,0.8215,0.400
et,Extra Trees Classifier,0.9104,0.9722,0.9184,0.9221,0.9199,0.8183,0.8191,0.630
rf,Random Forest Classifier,0.9028,0.9709,0.9116,0.9155,0.9130,0.8029,0.8043,0.648
lr,Logistic Regression,0.8890,0.9553,0.9022,0.9007,0.9008,0.7747,0.7763,0.054
ada,Ada Boost Classifier,0.8847,0.9514,0.8920,0.9018,0.8965,0.7664,0.7674,0.178
ridge,Ridge Classifier,0.8842,0.0000,0.8946,0.8992,0.8962,0.7653,0.7669,0.020
lda,Linear Discriminant Analysis,0.8842,0.9541,0.8946,0.8992,0.8962,0.7653,0.7669,0.044
svm,SVM - Linear Kernel,0.8804,0.0000,0.8971,0.8927,0.8935,0.7570,0.7602,0.036


In [ ]:
# tuned_best = [tune_model(i) for i in best]

In [ ]:
blend_3_soft = blend_models(estimator_list=best, method='soft', choose_better = True)
predict_model(blend_3_soft).head(10)

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.8875,0.9718,0.8881,0.9084,0.8981,0.7726,0.7728
1,0.9042,0.9789,0.9179,0.9111,0.9145,0.8055,0.8055
2,0.9208,0.9775,0.9328,0.9259,0.9294,0.8393,0.8394
3,0.9333,0.9842,0.9328,0.9470,0.9398,0.8651,0.8652
4,0.9458,0.9902,0.9552,0.9481,0.9517,0.8901,0.8901
5,0.9333,0.9807,0.9403,0.9403,0.9403,0.8648,0.8648
6,0.9417,0.9860,0.9333,0.9618,0.9474,0.8820,0.8825
7,0.9208,0.9797,0.9259,0.9328,0.9294,0.8393,0.8394
8,0.9167,0.9802,0.8889,0.9600,0.9231,0.8325,0.8354
9,0.9500,0.9875,0.9630,0.9489,0.9559,0.8982,0.8983


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.92,0.9808,0.9383,0.9157,0.9268,0.8386,0.8389


,Age,Flight Distance,Arrival Delay in Minutes,Gender_1,Customer Type_0,Type of Travel_1,Class_0,Class_1,Class_2,Seat comfort_1,Seat comfort_2,Seat comfort_3,Seat comfort_4,Seat comfort_5,Departure/Arrival time convenient_1,Departure/Arrival time convenient_2,Departure/Arrival time convenient_3,Departure/Arrival time convenient_4,Departure/Arrival time convenient_5,Gate location_1,Gate location_2,Gate location_3,Gate location_4,Gate location_5,Inflight wifi service_1,Inflight wifi service_2,Inflight wifi service_3,Inflight wifi service_4,Inflight wifi service_5,Inflight entertainment_1,Inflight entertainment_2,Inflight entertainment_3,Inflight entertainment_4,Inflight entertainment_5,Online support_1,Online support_2,Online support_3,Online support_4,Online support_5,Ease of Online booking_1,Ease of Online booking_2,Ease of Online booking_3,Ease of Online booking_4,Ease of Online booking_5,On-board service_1,On-board service_2,On-board service_3,On-board service_4,On-board service_5,Leg room service_0,Leg room service_1,Leg room service_2,Leg room service_3,Leg room service_4,Leg room service_5,Baggage handling_1,Baggage handling_2,Baggage handling_3,Baggage handling_4,Baggage handling_5,Checkin service_1,Checkin service_2,Checkin service_3,Checkin service_4,Checkin service_5,Cleanliness_1,Cleanliness_2,Cleanliness_3,Cleanliness_4,Cleanliness_5,Online boarding_1,Online boarding_2,Online boarding_3,Online boarding_4,Online boarding_5,target,Label,Score
0,0.303166,0.096516,-0.672116,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0,0,0.9952
1,-0.954282,-0.007025,-0.672116,1.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0,0,0.9384
2,-1.880823,-0.230967,-0.672116,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0,0.9505
3,-2.079368,-0.754853,2.049993,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1,1,0.5451
4,-1.219008,1.703806,-0.672116,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1,1,0.9988
5,-1.417553,0.023626,-0.672116,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1,1,0.9933
6,-0.093923,-2.164731,-0.672116,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1,0,0.6729
7,-0.888101,0.441022,-0.672116,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.

In [ ]:
final_blend_3_soft = finalize_model(blend_3_soft)
prediction = predict_model(final_blend_3_soft, data = test)

In [ ]:
submission = pd.read_csv("/content/drive/MyDrive/YBIGTA/DA 시니어 2차 프로젝트/airline_dataset/sample_submission.csv")
submission.head()

,id,target
0,1,0
1,2,0
2,3,0
3,4,0
4,5,0


In [ ]:
submission['target'] = prediction['Label']
submission.head()

,id,target
0,1,1
1,2,0
2,3,1
3,4,1
4,5,1


In [ ]:
submission.to_csv("/content/drive/MyDrive/YBIGTA/DA 시니어 2차 프로젝트/제출/성민/2.16_del_DC_not_tuned.csv",index=False)

## 모델학습

In [ ]:
import numpy as np

def ACCURACY(true, pred):   
    score = np.mean(true==pred)
    return score

In [ ]:
X_train = train.drop('target', axis=1)
y_train = train['target']
X_test = test.drop('target', axis=1)
y_test = test['target']


KeyError: ignored

In [ ]:
models = [
    XGBClassifier(eval_metric = 'logloss', \
                              max_depth = 5, \
                              min_child_weight = 3, \
                               gamma = 3, \
                               colsample_bytree = 0.5, \
                               n_estimators=700),
    LGBMClassifier(n_estimators=600, max_bin=400, num_leaves=24),
    CatBoostClassifier(silent=True, depth=6, l2_leaf_reg=7, learning_rate=0.1, n_estimators=500),
]

In [ ]:
for model in models:
    print(f'{type(model).__name__} score: {cross_val_score(model, X_train, y_train, cv=5,n_jobs=-1).mean()}')

In [ ]:
model_XGB = XGBClassifier(eval_metric='logloss', silent = True)

param_grid={'booster' :['gbtree'],
                 'silent':[True],
                 'max_depth':[5,6,8],
                 'min_child_weight':[1,3,5],
                 'gamma':[0,1,2,3],
                 'nthread':[4],
                 'colsample_bytree':[0.5,0.8],
                 'colsample_bylevel':[0.9],
                 'n_estimators':[50],
                 'objective':['binary:logistic'],
                 'random_state':[2]}

grid_cv_XGB=GridSearchCV(model_XGB, param_grid=param_grid, cv=5 , n_jobs=-1)
grid_cv_XGB.fit(X_train, y_train)
print('최적 하이퍼 파라미터: ', grid_cv_XGB.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv_XGB.best_score_))

최적 하이퍼 파라미터:  {'booster': 'gbtree', 'colsample_bylevel': 0.9, 'colsample_bytree': 0.5, 'gamma': 0, 'max_depth': 5, 'min_child_weight': 5, 'n_estimators': 50, 'nthread': 4, 'objective': 'binary:logistic', 'random_state': 2, 'silent': True}
최고 예측 정확도: 0.9230


In [ ]:
model_LGBM = LGBMClassifier()

gridParams = {
    'learning_rate': [0.005, 0.01],
    'n_estimators': [100, 500, 1000],
    'num_leaves': [12, 16, 20], # large num_leaves helps improve accuracy but might lead to over-fitting
    'boosting_type' : ['dart'], # for better accuracy -> try dart
    'objective' : ['binary'],
    'max_bin':[300, 600], # large max_bin helps improve accuracy but might slow down training progress
    }

grid_cv_LGBM = GridSearchCV(model_LGBM, param_grid=gridParams, cv=3 , n_jobs=-1)
grid_cv_LGBM.fit(X_train, y_train)
print('최적 하이퍼 파라미터: ', grid_cv_LGBM.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv_LGBM.best_score_))

최적 하이퍼 파라미터:  {'boosting_type': 'dart', 'learning_rate': 0.01, 'max_bin': 300, 'n_estimators': 1000, 'num_leaves': 16, 'objective': 'binary'}
최고 예측 정확도: 0.9007


In [ ]:
model_CAT = CatBoostClassifier(silent = True)

grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}

grid_cv_CAT = GridSearchCV(model_CAT, param_grid=grid, cv=3 , n_jobs=-1)
grid_cv_CAT.fit(X_train, y_train)
print('최적 하이퍼 파라미터: ', grid_cv_CAT.best_params_)
print('최고 예측 정확도: {:.4f}'.format(grid_cv_CAT.best_score_))

최적 하이퍼 파라미터:  {'depth': 6, 'l2_leaf_reg': 5, 'learning_rate': 0.03}
최고 예측 정확도: 0.9277


In [ ]:
ACCURACY(y_val, prediction)

0.935